## 1. Search for Granules

In [21]:
import os
import math
from urllib.parse import urlparse
from dateutil.parser import isoparse
import earthaccess as ea

def km_to_deg_lat(km: float) -> float:
    return km / 110.574

def km_to_deg_lon(km: float, lat_deg: float) -> float:
    return km / (111.320 * math.cos(math.radians(lat_deg)))

def point_radius_to_bbox(lon: float, lat: float, radius_km: float):
    dlat = km_to_deg_lat(radius_km)
    dlon = km_to_deg_lon(radius_km, lat)
    min_lon = max(-180.0, lon - dlon)
    max_lon = min( 180.0, lon + dlon)
    min_lat = max( -90.0, lat - dlat)
    max_lat = min(  90.0, lat + dlat)
    return (min_lon, min_lat, max_lon, max_lat)

def normalize_temporal(start_date: str, end_date: str):
    def to_iso(s, is_start):
        if "T" in s:
            return isoparse(s).strftime("%Y-%m-%dT%H:%M:%SZ")
        return f"{s}T00:00:00Z" if is_start else f"{s}T23:59:59Z"
    return (to_iso(start_date, True), to_iso(end_date, False))

def first_direct_link(g):
    """Return the first HTTPS direct data link (not OPeNDAP) if available."""
    try:
        links = g.data_links(access="direct")
    except Exception:
        links = getattr(g, "data_links", []) or []
    for href in links:
        h = str(href)
        if h.startswith("https://") and "opendap" not in h.lower():
            return h
    return None

def filename_from_link(href: str) -> str:
    if not href:
        return "UNKNOWN_FILENAME"
    return os.path.basename(urlparse(href).path)

def safe_umm(g):
    if hasattr(g, "umm"):
        return getattr(g, "umm") or {}
    try:
        return g.get("umm", {})  # type: ignore[attr-defined]
    except Exception:
        return {}

def safe_temporal(umm: dict):
    try:
        r = umm.get("TemporalExtent", {}).get("RangeDateTime", {})
        return r.get("BeginningDateTime", "N/A"), r.get("EndingDateTime", "N/A")
    except Exception:
        return "N/A", "N/A"

def safe_size_mb(umm: dict):
    try:
        adi = umm.get("DataGranule", {}).get("ArchiveAndDistributionInformation", [])
        if adi and "SizeMBDataGranule" in adi[0]:
            return float(adi[0]["SizeMBDataGranule"])
    except Exception:
        pass
    return None

def print_granule_list(results):
    print("\nFound granules:")
    for i, g in enumerate(results, start=1):
        href = first_direct_link(g)
        fname = filename_from_link(href)
        umm = safe_umm(g)
        t0, t1 = safe_temporal(umm)
        size = safe_size_mb(umm)
        size_str = f"{size:.1f} MB" if size is not None else "N/A"
        print(f"[{i:02d}] {fname} | {t0} → {t1} | Size: {size_str}")

def search_granules(
    lat: float,
    lon: float,
    start_date: str,
    end_date: str,
    search_radius_km: float = 1.0,
    product_short_name: str = "ECO_L2G_LSTE",
    product_version: str = "002",
    out_dir: str = "./ecostress_downloads",
):
    # Ensure output directory exists
    os.makedirs(out_dir, exist_ok=True)

    print("Logging in to Earthdata (interactive)…")
    ea.login(strategy="interactive")

    bbox = point_radius_to_bbox(lon, lat, search_radius_km)
    t0, t1 = normalize_temporal(start_date, end_date)

    print(f"\nSearching {product_short_name} v{product_version}")
    print(f"Spatial bbox: {bbox}")
    print(f"Temporal: {t0} to {t1}")

    results = ea.search_data(
        short_name=product_short_name,
        version=product_version,
        temporal=(t0, t1),
        bounding_box=bbox,
        provider="LPCLOUD",
        sort_key="-start_date",
    )

    if not results:
        print("No matching granules found.")
        return

    print_granule_list(results)

    sel = input("\nEnter the index of the granule to download: ").strip()
    try:
        idx = int(sel) - 1
        assert 0 <= idx < len(results)
    except Exception:
        print("Invalid selection.")
        return

    chosen = [results[idx]]

    print("\nDownloading…")
    paths = ea.download(chosen, out_dir)

    h5_path = "";
    if paths:
        print("Download complete:")
        for p in paths:
            print("  -", p)
            h5_path = p; #.split('\\')[1];
        return h5_path;
    else:
        print("Download returned no files. If you’re on Windows, ensure your browser login succeeded and try again.")
        


In [22]:
LAT = 38.92744661
LON = -106.95647367
START_DATE = "2020-08-16"
END_DATE   = "2020-08-18"


h5_path_txt = search_granules(
        LAT,
        LON,
        START_DATE,
        END_DATE,
        search_radius_km=1.0,
        # product_short_name, product_version, and out_dir use defaults
    )    


Logging in to Earthdata (interactive)…

Searching ECO_L2G_LSTE v002
Spatial bbox: (-106.96802094336361, 38.91840289267043, -106.94492639663638, 38.936490327329565)
Temporal: 2020-08-16T00:00:00Z to 2020-08-18T23:59:59Z

Found granules:
[01] UNKNOWN_FILENAME | 2020-08-18T20:46:48.667Z → 2020-08-18T20:47:40.636Z | Size: N/A
[02] UNKNOWN_FILENAME | 2020-08-18T14:16:22.287Z → 2020-08-18T14:17:14.257Z | Size: N/A
[03] UNKNOWN_FILENAME | 2020-08-17T21:34:34.694Z → 2020-08-17T21:35:26.663Z | Size: N/A
[04] UNKNOWN_FILENAME | 2020-08-17T21:33:42.724Z → 2020-08-17T21:34:34.693Z | Size: N/A
[05] UNKNOWN_FILENAME | 2020-08-17T15:04:27.032Z → 2020-08-17T15:05:19.002Z | Size: N/A
[06] UNKNOWN_FILENAME | 2020-08-17T15:03:35.061Z → 2020-08-17T15:04:27.032Z | Size: N/A
[07] UNKNOWN_FILENAME | 2020-08-16T22:21:55.918Z → 2020-08-16T22:22:47.888Z | Size: N/A



Enter the index of the granule to download:  5



Downloading…


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Download complete:
  - ecostress_downloads\ECOv002_L2G_LSTE_11997_006_20200817T150427_0712_01.h5


In [19]:
print(h5_path_txt)

ECOv002_L2G_LSTE_11997_006_20200817T150427_0712_01.h5


## 2. Download Image (celsius)

In [23]:
import os
import numpy as np
from osgeo import gdal

def h5_to_geotiff(
    h5_path: str,
    out_tif_c: str = None,
):
    """Convert ECOSTRESS LST HDF5 subdataset to GeoTIFF (Celsius)."""

    if not os.path.exists(h5_path):
        raise FileNotFoundError(f"HDF5 file not found: {h5_path}")

    # If output not provided, put next to input
    if out_tif_c is None:
        base = os.path.splitext(h5_path)[0]
        out_tif_c = base + "_lst_celsius.tif"

    # Ensure output directory exists
    os.makedirs(os.path.dirname(out_tif_c), exist_ok=True)

    # 1) Find the LST subdataset
    g = gdal.Open(h5_path, gdal.GA_ReadOnly)
    if g is None:
        raise RuntimeError("GDAL could not open the HDF5 file.")
    subs = g.GetSubDatasets()

    lst_sds = None
    for name, desc in subs:
        low = (name + " " + desc).lower()
        if "/lst" in low and "qc" not in low:  # pick LST, not QC
            lst_sds = name
            break
    if not lst_sds:
        raise RuntimeError("LST subdataset not found. Run gdalinfo on the .h5 and share the output.")

    # 2) Open LST subdataset and read data (Kelvin)
    src = gdal.Open(lst_sds, gdal.GA_ReadOnly)
    arr_k = src.ReadAsArray().astype("float32")

    # 3) Mask fill values (common ECOSTRESS fill)
    for fv in (-9999.0, -9998.0):
        arr_k[arr_k == fv] = np.nan

    # 4) Convert to Celsius
    arr_c = arr_k - 273.15

    # 5) Write GeoTIFF with same georeferencing
    driver = gdal.GetDriverByName("GTiff")
    dst = driver.Create(
        out_tif_c,
        src.RasterXSize,
        src.RasterYSize,
        1,
        gdal.GDT_Float32,
        options=["TILED=YES", "COMPRESS=DEFLATE"]
    )
    dst.SetGeoTransform(src.GetGeoTransform())
    dst.SetProjection(src.GetProjection())
    band = dst.GetRasterBand(1)
    band.WriteArray(arr_c)
    band.SetNoDataValue(np.nan)
    band.FlushCache()
    dst = None
    src = None

    print("Wrote:", out_tif_c)
    return out_tif_c


In [24]:
# H5_PATH = r"ecostress_downloads\ECOv002_L2G_LSTE_39343_012_20250614T104308_0713_01.h5"
H5_PATH = h5_path_txt

h5_to_geotiff(H5_PATH)

Wrote: ecostress_downloads\ECOv002_L2G_LSTE_11997_006_20200817T150427_0712_01_lst_celsius.tif


'ecostress_downloads\\ECOv002_L2G_LSTE_11997_006_20200817T150427_0712_01_lst_celsius.tif'